### **Train and Test the Models on the GTSRB dataset**

Three models for traffic sign classification:
1. CNNsmall (original LISA-CNN)
2. CNNlarge (original GTSRB-CNN)
3. CNN-STN 

Five generic image classification models
1. ResNet18
2. EfficientNet-B0
3. DenseNet-121
4. MobileNetv2
5. ShuffleNetv2

In [1]:
import pickle

with open('dataset/GTSRB/train.pkl', 'rb') as f:
    train = pickle.load(f)
    train_data, train_labels = train['data'], train['labels']
with open('dataset/GTSRB/test.pkl', 'rb') as f:
    test = pickle.load(f)
    test_data, test_labels = test['data'], test['labels']

# Calculate the sizes in percentage
total_size = len(train_data) + len(test_data)
train_percentage = (len(train_data) / total_size) * 100
test_percentage = (len(test_data) / total_size) * 100

print(f"Training data size: {len(train_data)} which is {train_percentage:.2f}% of the total dataset")
print(f"Test data size: {len(test_data)} which is {test_percentage:.2f}% of the total dataset")

Training data size: 39209 which is 75.64% of the total dataset
Test data size: 12630 which is 24.36% of the total dataset


In [2]:
import pickle
import os
import json
from models import *
from utils import *
from torchvision.models import mobilenet_v2, efficientnet_b0, densenet121, shufflenet_v2_x1_0

# gtsrb dataset
with open('classes.json', 'r') as config:
    params = json.load(config)
    class_n = params['GTSRB']['class_n']
    device = params['device']



def train_model(model_name, adv_train=False):

    with open('dataset/GTSRB/train.pkl', 'rb') as f:
        train = pickle.load(f)
        train_data, train_labels = train['data'], train['labels']
    with open('dataset/GTSRB/test.pkl', 'rb') as f:
        test = pickle.load(f)
        test_data, test_labels = test['data'], test['labels']

    print('Loading .pkl done.')

    processed_train = np.array([
        pre_process_image(train_data[i]) for i in range(len(train_data))],
        dtype=np.float32) if not adv_train else train_data

    print('Processing train data done.')

    processed_test = np.array([
        pre_process_image(test_data[i]) for i in range(len(test_data))],
        dtype=np.float32)

    print('Processing test data done.')

    augment_data_train, augment_data_labels = gen_extra_data(
        train_data, train_labels, 10, 30, 5, 5, 1, preprocess=not adv_train)

    print('Augmentation process done.')

    image_train = np.concatenate([processed_train, augment_data_train], 0)
    label_train = np.concatenate([train_labels, augment_data_labels], 0)
    image_test, label_test = processed_test, test_labels

    if model_name=='TransformerGTSRB':
        training_model = Transformer(class_n=class_n).to(device).apply(weights_init)
    elif model_name=='CNNsmallGTSRB':
        training_model = CNNsmall(class_n=class_n).to(device).apply(weights_init)
    elif model_name=='CNNlargeGTSRB':
        training_model = CNNlarge(class_n=class_n).to(device).apply(weights_init)
    elif model_name=='ResNet18GTSRB':
        training_model = ResNet18(class_n=class_n).to(device) #.apply(weights_init)
    elif model_name=='MobileNetv2GTSRB':
        training_model = mobilenet_v2(num_classes=class_n).to(device)
    elif model_name=='EfficientNetGTSRB':
        training_model = efficientnet_b0(num_classes=class_n).to(device) #.apply(weights_init)
    elif model_name=='DenseNet121GTSRB':
        training_model = densenet121(num_classes=class_n).to(device) #.apply(weights_init)
    elif model_name=='ShuffleNetv2x1GTSRB':
        training_model = shufflenet_v2_x1_0(num_classes=class_n).to(device) #.apply(weights_init)

    print('Loading model done.')
    training(training_model=training_model,
             train_data=image_train,
             train_labels=label_train,
             test_data=image_test,
             test_labels=label_test,
             model_name=model_name,
             device=device)

def test_model(model_name):

    if model_name=='TransformerGTSRB':
        trained_model = Transformer(class_n=class_n).to(device)
    elif model_name=='CNNsmallGTSRB':
        trained_model = CNNsmall(class_n=class_n).to(device)
    elif model_name=='CNNlargeGTSRB':
        trained_model = CNNlarge(class_n=class_n).to(device)
    elif model_name=='ResNet18GTSRB':
        trained_model = ResNet18(class_n=class_n).to(device)
    elif model_name=='MobileNetv2GTSRB':
        trained_model = mobilenet_v2(num_classes=class_n).to(device)
    elif model_name=='EfficientNetGTSRB':
        trained_model = efficientnet_b0(num_classes=class_n).to(device)
    elif model_name=='DenseNet121GTSRB':
        trained_model = densenet121(num_classes=class_n).to(device)
    elif model_name=='ShuffleNetv2x1GTSRB':
        trained_model = shufflenet_v2_x1_0(num_classes=class_n).to(device)


    trained_model.load_state_dict(
        torch.load(f'models/{model_name}.pth',
                   map_location=torch.device(device)))

    with open('dataset/GTSRB/test.pkl', 'rb') as f:
        test = pickle.load(f)
        test_data, test_labels = test['data'], test['labels']

    test_data = np.array([
        pre_process_image(test_data[i]) for i in range(len(test_data))],
        dtype=np.float32)

    test_set = TrafficSignDataset(test_data, test_labels, device)
    test_loader = DataLoader(test_set, batch_size=64, shuffle=False)

    trained_model.eval()
    with torch.no_grad():
        test_acc, _, inference_time = model_epoch(trained_model, test_loader)

    # Calculate average inference time per sample
    avg_inference_time = inference_time / len(test_set)

    print(f'Test Acc: {round(float(test_acc / test_set.__len__()), 5)}')
    print(f'Average Inference Time: {avg_inference_time * 1000.0:.5f} ms')


##### **Training**

In [3]:
# train_model(model_name='CNNlargeGTSRB')
# train_model(model_name='ShuffleNetv2x1GTSRB')
# train_model(model_name='CNNsmallGTSRB')

In [4]:
test_model(model_name='TransformerGTSRB')

/fzi/ids/pavlitsk/.local/lib/python3.8/site-packages/torch/nn/functional.py:4255: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  warnings.warn(
/fzi/ids/pavlitsk/.local/lib/python3.8/site-packages/torch/nn/functional.py:4193: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  warnings.warn(


Test Acc: 0.9943
Average Inference Time: 0.01249 ms


In [5]:
test_model(model_name='CNNsmallGTSRB')

Test Acc: 0.98124
Average Inference Time: 0.00550 ms


In [6]:
test_model(model_name='CNNlargeGTSRB')

Test Acc: 0.98907
Average Inference Time: 0.01084 ms


In [7]:
test_model(model_name='ResNet18GTSRB')

Test Acc: 0.99177
Average Inference Time: 0.02686 ms


In [8]:
test_model(model_name='MobileNetv2GTSRB')

Test Acc: 0.98361
Average Inference Time: 0.05126 ms


In [9]:
test_model(model_name='EfficientNetGTSRB')

Test Acc: 0.98725
Average Inference Time: 0.08161 ms


In [10]:
test_model(model_name='DenseNet121GTSRB')

Test Acc: 0.98092
Average Inference Time: 0.15402 ms


In [11]:
test_model(model_name='ShuffleNetv2x1GTSRB')

Test Acc: 0.96057
Average Inference Time: 0.06554 ms


In [12]:
from torchinfo import summary

model = efficientnet_b0(class_n=class_n)
pytorch_total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(pytorch_total_params)

5288548
